It runs and moves in order from the top.
Settings include model changes.
>上から順に実行して動きます。
>設定はモデル変更などがあります。

Adapted from（参考元）: 

https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb

In [ ]:
#@title Clone webUI repository.（webUIリポジトリのクローン）
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui --quiet
%cd /content/stable-diffusion-webui

In [ ]:
#@title Download the model from Hugging Face.（Hugging Faceからモデルをダウンロード）

models_dict = {
  "trinart-characters-19m" : "https://huggingface.co/naclbit/trinart_characters_19.2m_stable_diffusion_v1/blob/main/trinart_characters_it4_v1.ckpt",
  "trinart2-step115000" : "https://huggingface.co/naclbit/trinart_stable_diffusion_v2/blob/main/trinart2_step115000.ckpt",
  "trinart2-step95000" : "https://huggingface.co/naclbit/trinart_stable_diffusion_v2/blob/main/trinart2_step95000.ckpt",
  "trinart2-step60000" : "https://huggingface.co/naclbit/trinart_stable_diffusion_v2/blob/main/trinart2_step60000.ckpt",
  "waifu-v13-penultimate" : "https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/models/wd-1-3-penultimate-ucg-cont.ckpt",
  "waifu-v13-float32" : "https://huggingface.co/hakurei/waifu-diffusion-v1-3/blob/main/wd-v1-3-float32.ckpt",
  "waifu-v13-float16" : "https://huggingface.co/hakurei/waifu-diffusion-v1-3/blob/main/wd-v1-3-float16.ckpt",
  "waifu-v12-prune" : "https://huggingface.co/crumb/pruned-waifu-diffusion/blob/main/model-pruned.ckpt",
  "stable-v15" : "https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.ckpt",
  "stable-v15-inpainting" : "https://huggingface.co/runwayml/stable-diffusion-inpainting/blob/main/sd-v1-5-inpainting.ckpt",
  "stable-v14" : "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/blob/main/sd-v1-4.ckpt",
  "naruto-diffusers" : "https://huggingface.co/lambdalabs/sd-naruto-diffusers",
  "pokemon-diffusers" : "https://huggingface.co/lambdalabs/sd-pokemon-diffusers",
  "robo-v1" : "https://huggingface.co/nousr/robo-diffusion/blob/main/models/robo-diffusion-v1.ckpt",
  "eva-v22-not-hd" : "https://huggingface.co/crumb/eva-model-ckpt/blob/main/eva-fusion-v2.22-notHD.ckpt",
  "eva-v21" : "https://huggingface.co/crumb/eva-model-ckpt/blob/main/eva-fusion-v2.1.ckpt",
  "icon" : "https://huggingface.co/crumb/icon-diffusion-ckpt/blob/main/icon-diffusion.ckpt",
  "pixel-art-sprite-sheet-v1___PixelartFSS__PixelartRSS__PixelartBSS__PixelartLSS" : "https://huggingface.co/Onodofthenorth/SD_PixelArt_SpriteSheet_Generator/blob/main/PixelartSpritesheet_V.1.ckpt",
  "deathnote-v2___DNStyle_style__Light_Yagami_man__Ryuk_demon" : "https://huggingface.co/Guizmus/DeathNote/blob/main/DeathNote_v2_4k.ckpt",
  "arcane-v3___arcane_style" : "https://huggingface.co/nitrosocke/Arcane-Diffusion/blob/main/arcane-diffusion-v3.ckpt",
  "arcane-v2___arcane_style" : "https://huggingface.co/nitrosocke/Arcane-Diffusion/blob/main/arcane-diffusion-v2.ckpt",
  "eldenring-v3___elden_ring_style" : "https://huggingface.co/nitrosocke/elden-ring-diffusion/blob/main/eldenRing-v3-pruned.ckpt",
  "eldenring-v2___elden_ring_style" : "https://huggingface.co/nitrosocke/elden-ring-diffusion/blob/main/eldenring-v2-pruned.ckpt",
  "eldenring-v1___elden_ring_style" : "https://huggingface.co/nitrosocke/elden-ring-diffusion/blob/main/eldenring-v1-pruned.ckpt",
  "spiderverse-v1___spiderverse_style" : "https://huggingface.co/nitrosocke/spider-verse-diffusion/blob/main/spiderverse-v1-pruned.ckpt",
  "archer-v1___archer_style" : "https://huggingface.co/nitrosocke/archer-diffusion/blob/main/archer-v1.ckpt",
  "modern-disney-v1___modern_disney_style" : "https://huggingface.co/nitrosocke/mo-di-diffusion/blob/main/modernDisney-v1-pruned.ckpt",
  "cyberpunk-anime___dgs_illustration_style" : "https://huggingface.co/DGSpitzer/Cyberpunk-Anime-Diffusion/blob/main/Cyberpunk-Anime-Diffusion.ckpt",
  "tohoku-v1___itako__zunko__kiritan__zundamon__metan" : "https://huggingface.co/shirayu/sd-tohoku-v1/blob/main/sd-tohoku-v1.model.ckpt",
  "ff14-v1___xivcine_person" : "https://huggingface.co/herpritts/xivcine_person_v1/blob/main/xivcine_person_v1.ckpt",
  "tron-legacy___trnlgcy" : "https://huggingface.co/dallinmackay/Tron-Legacy-diffusion/blob/main/Tron-Legacy-Style-trnlgcy.ckpt",
  "classic-anim___classic_disney_style": "https://huggingface.co/nitrosocke/classic-anim-diffusion/blob/main/classicAnim-v1.ckpt",
  "poolsuite___poolsuite_style" : "https://huggingface.co/prompthero/poolsuite-diffusion/blob/main/poolsuite_step2010.ckpt",
}

vaes_dict = {
  "None" : "",
  "trinart-characters-19m" : "https://huggingface.co/naclbit/trinart_characters_19.2m_stable_diffusion_v1/blob/main/autoencoder_fix_kl-f8-trinart_characters.ckpt",
  "waifu-v14-anime2" : "https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/vae/kl-f8-anime2.ckpt",
  "waifu-v14-anime1" : "https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/vae/kl-f8-anime.ckpt",
  "sd-vae-ft-mse" : "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.ckpt",
  "sd-vae-ft-ema" : "https://huggingface.co/stabilityai/sd-vae-ft-ema-original/blob/main/vae-ft-ema-560000-ema-pruned.ckpt",
}

textual_inversion_embeddings_dict = {
  "None" : "",
  "toho-pixel" : "https://huggingface.co/sd-concepts-library/toho-pixel/blob/main/learned_embeds.bin",
  "pokemon-conquest" : "https://huggingface.co/sd-concepts-library/pokemon-conquest-sprites/blob/main/learned_embeds.bin",
  "gba-pokemon" : "https://huggingface.co/sd-concepts-library/gba-pokemon-sprites/blob/main/learned_embeds.bin",
}

hypernetworks_dict = {
  "None" : "",
  "zelda" : "https://huggingface.co/wavymulder/zelda-diffusion-HN/blob/main/zeldaBOTW.pt",
}

#@markdown get a token from Hugging Face. https://huggingface.co/settings/tokens （Hugging Faceからトークンを取得）
user_token = "" #@param {type:"string"}
model = "trinart-characters-19m" #@param ['trinart-characters-19m', 'trinart2-step115000', 'trinart2-step95000', 'trinart2-step60000', 'waifu-v13-penultimate', 'waifu-v13-float32', 'waifu-v13-float16', 'waifu-v12-prune', 'stable-v15', 'stable-v15-inpainting', 'stable-v14', 'naruto-diffusers', 'pokemon-diffusers', 'robo-v1', 'eva-v22-not-hd', 'eva-v21', 'icon', 'pixel-art-sprite-sheet-v1___PixelartFSS__PixelartRSS__PixelartBSS__PixelartLSS', 'deathnote-v2___DNStyle_style__Light_Yagami_man__Ryuk_demon', 'arcane-v3___arcane_style', 'arcane-v2___arcane_style', 'eldenring-v3___elden_ring_style', 'eldenring-v2___elden_ring_style', 'eldenring-v1___elden_ring_style', 'spiderverse-v1___spiderverse_style', 'archer-v1___archer_style', 'modern-disney-v1___modern_disney_style', 'cyberpunk-anime___dgs_illustration_style', 'tohoku-v1___itako__zunko__kiritan__zundamon__metan', 'ff14-v1___xivcine_person', 'tron-legacy___trnlgcy', 'classic-anim___classic_disney_style', 'poolsuite___poolsuite_style']
vae = "None" #@param ['None', 'trinart-characters-19m', 'waifu-v14-anime2', 'waifu-v14-anime1', 'sd-vae-ft-mse', 'sd-vae-ft-ema']
textual_inversion_embedding = "None" #@param ['None', 'toho-pixel', 'pokemon-conquest', 'gba-pokemon']
hypernetwork = "None" #@param ['None', 'zelda']
if user_token != "":
  user_header = f"\"Authorization: Bearer {user_token}\""
else:
  user_header = ""

!wget -q -O convert_diffusers_to_original_stable_diffusion.py https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_diffusers_to_original_stable_diffusion.py

def download_diffusers(name, url):
  !git lfs clone {url} repositories/{name}
  !python convert_diffusers_to_original_stable_diffusion.py \
    --model_path repositories/{name} \
    --checkpoint_path models/Stable-diffusion/{name}.ckpt \
    --half

if model.endswith("diffusers"):
  download_diffusers(model, models_dict[model])
else:
  model_url = models_dict[model].replace("/blob/", "/resolve/")
  !wget --header={user_header} {model_url} -O /content/stable-diffusion-webui/models/Stable-diffusion/{model}.ckpt

if vae != 'None':
  vae_url = vaes_dict[vae].replace("/blob/", "/resolve/")
  !wget --header={user_header} {vae_url} -O /content/stable-diffusion-webui/models/VAE/{vae}.vae.pt

if textual_inversion_embedding != 'None':
  textual_inversion_embedding_url = textual_inversion_embeddings_dict[textual_inversion_embedding].replace("/blob/", "/resolve/")
  !wget --header={user_header} {textual_inversion_embedding_url} -O /content/stable-diffusion-webui/embeddings/{textual_inversion_embedding}.pt

if hypernetwork != 'None':
  import os
  os.makedirs("/content/stable-diffusion-webui/models/hypernetworks/", exist_ok=True)
  hypernetwork_url = hypernetworks_dict[hypernetwork].replace("/blob/", "/resolve/")
  !wget --header={user_header} {hypernetwork_url} -O /content/stable-diffusion-webui/models/hypernetworks/{hypernetwork}.pt
  
!wget -q https://raw.githubusercontent.com/thx-pw/stable-diffusion-webui-colab/main/styles.csv -O /content/stable-diffusion-webui/styles.csv

localization = "ja_JP" #@param ['None', 'ar_AR', 'de_DE', 'es_ES', 'fr-FR', 'it_IT', 'ja_JP', 'ko_KR', 'pt_BR', 'ru_RU', 'tr_TR', 'zh_CN', 'zh_TW']
!echo '{"localization":' > "/content/stable-diffusion-webui/config.json"
!echo '"'{localization}'"' >> "/content/stable-diffusion-webui/config.json"
!echo '}' >> "/content/stable-diffusion-webui/config.json"

In [ ]:
#@title Download the model from Google Drive.（Google Driveからモデルをダウンロード）

import os
import glob
from google.colab import drive # type: ignore

use_google_drive = False #@param {type:"boolean"}
sd_webui_gdrive = "/content/drive/MyDrive/sd/stable-diffusion-webui" #@param {type:"string"}
sd_webui_colab = "/content/stable-diffusion-webui"

create_directory_gdrive = False #@param {type:"boolean"}
#@markdown Find and copy the following files from the stable-diffusion-webui(sd_webui_gdrive) directory.
#@markdown > stable-diffusion-webui(sd_webui_gdrive)ディレクトリから以下のファイルを見つけてコピーします。

#@markdown + models/Stable-diffusion/\*.ckpt
#@markdown + models/VAE/\*.ckpt
#@markdown + models/VAE/\*.pt
#@markdown + models/Stable-diffusion/\*.vae.ckpt
#@markdown + models/Stable-diffusion/\*.vae.pt
#@markdown + embeddings/\*.pt
#@markdown + embeddings/\*.bin
#@markdown + models/hypernetworks/*.pt
#@markdown + styles.csv
#@markdown + config.json
#@markdown + ui-config.json
#@markdown + /content/drive/MyDrive/model.ckpt
#@markdown + /content/drive/MyDrive/AI/models/*.ckpt

model_checkpoints = "models/Stable-diffusion"
model_model_vaes = "models/Stable-diffusion"
model_vae_vaes = "models/VAE"
textual_inversion_embeddings = "embeddings"
hypernetworks = "models/hypernetworks"
config_list = ["styles.csv", "config.json", "ui-config.json"]

model_checkpoints_gdrive = os.path.join(sd_webui_gdrive, model_checkpoints)
model_model_vaes_gdrive = os.path.join(sd_webui_gdrive, model_model_vaes)
model_vae_vaes_gdrive = os.path.join(sd_webui_gdrive, model_vae_vaes)
textual_inversion_embeddings_gdrive = os.path.join(sd_webui_gdrive, textual_inversion_embeddings)
hypernetworks_gdrive = os.path.join(sd_webui_gdrive, hypernetworks)

model_checkpoints_colab = os.path.join(sd_webui_colab, model_checkpoints)
model_model_vaes_colab = os.path.join(sd_webui_colab, model_model_vaes)
model_vae_vaes_colab = os.path.join(sd_webui_colab, model_vae_vaes)
textual_inversion_embeddings_colab = os.path.join(sd_webui_colab, textual_inversion_embeddings)
hypernetworks_colab = os.path.join(sd_webui_colab, hypernetworks)

if use_google_drive:
  drive.mount("/content/drive")

  ckpt_path = "/content/drive/MyDrive/model.ckpt"
  if os.path.exists(ckpt_path):
    !cp {ckpt_path} {model_checkpoints_colab}
  for model_checkpoint in glob.glob(f"/content/drive/MyDrive/AI/models/*.ckpt"):
    !cp {model_checkpoint} {model_checkpoints_colab}
 
  for config in config_list:
    config_gdrive = os.path.join(sd_webui_gdrive, config)
    if os.path.exists(config_gdrive):
      !cp {config_gdrive} {sd_webui_colab}

  if create_directory_gdrive:
    os.makedirs(model_checkpoints_gdrive, exist_ok=True)
    os.makedirs(model_model_vaes_gdrive, exist_ok=True)
    os.makedirs(model_vae_vaes_gdrive, exist_ok=True)
    os.makedirs(textual_inversion_embeddings_gdrive, exist_ok=True)
    os.makedirs(hypernetworks_gdrive, exist_ok=True)
  os.makedirs(hypernetworks_colab, exist_ok=True)

  for model_checkpoint in glob.glob(f"{model_checkpoints_gdrive}/*.ckpt"):
    !cp {model_checkpoint} {model_checkpoints_colab}

  files = glob.glob(f"{model_model_vaes_gdrive}/*.vae.ckpt")
  files.extend(glob.glob(f"{model_model_vaes_gdrive}/*.vae.pt"))
  for model_vae in files:
    !cp {model_vae} {model_model_vaes_colab}
    
  files = glob.glob(f"{model_vae_vaes_gdrive}/*.ckpt")
  files.extend(glob.glob(f"{model_vae_vaes_gdrive}/*.pt"))
  for model_vae in files:
    !cp {model_vae} {model_vae_vaes_colab}

  files = glob.glob(f"{textual_inversion_embeddings_gdrive}/*.pt")
  files.extend(glob.glob(f"{textual_inversion_embeddings_gdrive}/*.bin"))
  for textual_inversion_embedding in files:
    !cp {textual_inversion_embedding} {textual_inversion_embeddings_colab}

  for hypernetwork in glob.glob(f"{hypernetworks_gdrive}/*.pt"):
    !cp {hypernetwork} {hypernetworks_colab}
   

In [ ]:
#@title Installing Requirements from https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py

!git clone https://github.com/timojl/clipseg repositories/clipseg
!wget -q -O scripts/txt2mask.py https://raw.githubusercontent.com/ThereforeGames/txt2mask/main/scripts/txt2mask.py
!wget -q -O scripts/moisaic.py https://raw.githubusercontent.com/Pfaeff/sd-web-ui-scripts/main/moisaic.py
!wget -q -O scripts/sd_upscale_interrogate.py https://raw.githubusercontent.com/thx-pw/stable-diffusion-webui-colab/main/scripts/sd_upscale_interrogate.py

!wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.1
!wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.2
!mv Dependencies_AUT.1 Dependencies_AUT.7z.001
!mv Dependencies_AUT.2 Dependencies_AUT.7z.002
!7z x Dependencies_AUT.7z.001

!cp -r ./usr/local/lib/python3.7/dist-packages /usr/local/lib/python3.7/
!rm -r ./usr
!rm Dependencies_AUT.7z.001
!rm Dependencies_AUT.7z.002
  
!wget -q -O repositories/stable-diffusion/ldm/modules/attention.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/attention.py

!pip install -qq scikit-image --upgrade
!pip install -qq tensorflow tensorflow-io
!pip install -qq git+https://github.com/KichangKim/DeepDanbooru.git@edf73df4cdaeea2cf00e9ac08bd8a9026b7a7b26#egg=deepdanbooru

import time
from re import search
from subprocess import getoutput
from IPython.display import clear_output

def xformers() -> None:
  smi_out = getoutput('nvidia-smi')
  supported = search('(T4|P100|V100|A100|K80)', smi_out)

  if not supported:
    while True:
      print("\x1b[1;31mThe current GPU is not supported, try starting a new session.\x1b[0m")
      time.sleep(5)
  else:
    supported = supported.group(0)

  !pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/{supported}/xformers-0.0.13.dev0-py3-none-any.whl

  clear_output()
  print('[1;32mDone!')

xformers()


In [ ]:
#@title Launch the webUI.（webUIの起動）

username = "user" #@param {type:"string"}
password = "pass" #@param {type:"string"}
tunnel = "gradio" #@param ["ngrok", "gradio"]
#@markdown get a token from for ngrok. https://dashboard.ngrok.com/get-started/your-authtoken （ngrokからトークンを取得）
ngrok_authtoken = "" #@param {type:"string"}
ngrok_region = "jp" #@param ["us", "eu", "au", "ap", "sa", "jp", "in"]


common_args = f"--enable-insecure-extension-access --disable-safe-unpickle --update-check --deepdanbooru --gradio-debug --gradio-auth {username}:{password}"
if tunnel == "ngrok":
  !COMMANDLINE_ARGS="{common_args} --ngrok {ngrok_authtoken} --ngrok-region {ngrok_region}" REQS_FILE="requirements.txt" python launch.py
else:
  !COMMANDLINE_ARGS="{common_args} --share" REQS_FILE="requirements.txt" python launch.py


In [ ]:
#@title Save to zip file with same password as cell above. （上のセルと同じパスワードでzipファイルに保存）

%cd /content/stable-diffusion-webui
!zip -er --password={password} outputs.zip outputs 

In [ ]:
#@title Save the zip file to Google Drive. （Google Driveにzipファイルを保存）
from google.colab import drive 
drive_path = "/content/drive"
drive.mount(drive_path, force_remount=False)

!cp "/content/stable-diffusion-webui/outputs.zip" "/content/drive/MyDrive"